# 01. Setup and Load Data

### 1.1 Dependencies Install and Setup

In [3]:
#!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 6.8 MB/s eta 0:00:00a 0:00:01


In [1]:
import numpy as np
import cv2
import os
import pandas as pd
from keras.models import load_model
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
from mtcnn.mtcnn import MTCNN

2024-02-03 11:41:41.975309: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 1.2 Data Loading

In [2]:
current_directory = os.getcwd()
target_folder = os.path.join(current_directory, "studio_folder_check")
target_folder2 = os.path.join(target_folder, "product_ref_call")

##### I will use the xlsx file for references in the studio folder to check the model performance on the 70+ images that i know what the result should be.

In [3]:
product_ref_path = os.path.join(target_folder2, "product_ref.xlsx")

In [4]:
if not os.path.exists(target_folder2):
    print("File Error: The file does not exist.", product_ref_path)
    exit()
try:
    product_ref_df = pd.read_excel(product_ref_path)
except Exception as e:
    print("Error reading the Excel file:", e)
    exit()

In [5]:
product_ref_df

,product_ref
0,216786_FU
1,217013_FU
2,217013_BK
3,216855_KK
4,218093_BK
5,216943_BM
6,217022_GY
7,216394_BM
8,216875_LG
9,216875_FU


In [6]:
target_folder = os.path.join(current_directory, "studio_folder_check")
target_folder2 = os.path.join(target_folder, "product_ref_call")
downloaded_images_path = os.path.join(target_folder, "studio_folder")

##### DF to store the results of the predictions based on the image file.

In [7]:
images_to_check = set(os.listdir(downloaded_images_path))
images_to_check = pd.DataFrame(images_to_check)
images_to_check.rename(columns={0: "image_file_name"}, inplace=True)
images_to_check["model_images_predictions"] = np.nan

images_to_check

,image_file_name,model_images_predictions
0,216780_FU_3y.jpg,NaN
1,218093_BK_2y.jpg,NaN
2,216875_LG_3y.jpg,NaN
3,216875_FU_5y.jpg,NaN
4,217013_BK_5y.jpg,NaN
...,...,...
70,218093_BK_1yf.jpg,NaN
71,217022_GY_2yf.jpg,NaN
72,216855_KK_8y.jpg,NaN
73,216875_FU_2y.jpg,NaN


# 02. First Test With Small Dataset

### 2.1 Model Run (No Param. Tuning)

In [8]:
parfois_eye_detector = MTCNN()

##### With no changes on the initial and standard parameters.

#### Version Zero

In [9]:
def model_predictions(row):
    img_file_name = row["image_file_name"]
    img_path = os.path.join(downloaded_images_path, img_file_name)
    img_to_check = cv2.imread(img_path)
    
    if img_to_check is not None:
        faces = parfois_eye_detector.detect_faces(img_to_check)
        
        if faces:
            return 1
        else:
            return 0
    else:
        print(f"{img_file_name}: Unable to read the image")
        return None
    

In [10]:
images_to_check["model_images_predictions"] = images_to_check.apply(model_predictions, axis=1)

1/1 [==============================] - 0s 20ms/step


In [12]:
images_to_check = images_to_check.sort_values(by="image_file_name").reset_index(drop=True)
target_folder3 = os.path.join(target_folder, "studio_folder_true.xlsx")
studio_folder_true = pd.read_excel(target_folder3)
results_check = (images_to_check["model_images_predictions"] != studio_folder_true["model_images_check"]).sum()
results_check

12

##### After the check with another xlsx file that has the correct outputs the model got 12 results wrong out of 75. A good start.

### 2.2 Model Run (With Param. Tuning)

In [17]:
images_to_check.rename(columns={0: "image_file_name"}, inplace=True)
images_to_check["model_images_predictions"] = np.nan

#### Version one

##### I set up the min_face_size to 30 pixels, meaning that faces smaller then 30 will not be considered. Also works better for images of a full body, as we expect the face to be smaller. 

##### The thresholds steps are for the P-Net, R-Net and O-net. They work in a first to last and the threshold helps refine the results that are passed from P-Net to R-Net and from R-Net to O-Net.

In [13]:
parfois_eye_detector_2 = MTCNN(min_face_size=30, 
                               steps_threshold=[0.7, 0.8, 0.8])

def model_predictions_2(row):
    img_file_name = row["image_file_name"]
    img_path = os.path.join(downloaded_images_path, img_file_name)
    img_to_check = cv2.imread(img_path)
    
    if img_to_check is not None:
        faces = parfois_eye_detector_2.detect_faces(img_to_check)
        
        if faces:
            return 1
        else:
            return 0
    else:
        print(f"{img_file_name}: Unable to read the image")
        return None
    
images_to_check["model_images_predictions"] = images_to_check.apply(model_predictions_2, axis=1)

1/1 [==============================] - 0s 20ms/step


In [14]:
studio_folder_true = pd.read_excel(target_folder3)
results_check = (images_to_check["model_images_predictions"] != studio_folder_true["model_images_check"]).sum()
results_check

10

##### After the check i got 10 wrong outputs from 75. Its performing better.

In [18]:
images_to_check["model_images_predictions"] = np.nan

### 2.3 Model Run (With Param. Tuning and Keypoint targeting)

#### Version two

##### The output of the model is not 0 or 1 but a dictionary like this:

##### {'box': [1942, 716, 334, 415], 'confidence': 0.9999997615814209, 'keypoints': {'left_eye': (2053, 901), 'right_eye': (2205, 897), 'nose': (2139, 976), 'mouth_left': (2058, 1029), 'mouth_right': (2206, 1023)}}

##### This are the coordinates on the image for the different components of the face. For Parfois, to be a image with full recognition model that image need to have eyes and mouth, but it can also be rotated. We can extract the info from the keypoints and set up a if/else condition.

In [15]:
parfois_eye_detector_2 = MTCNN(min_face_size=30, 
                               steps_threshold=[0.7, 0.8, 0.8])

def model_predictions_2(row):
    img_file_name = row["image_file_name"]
    img_path = os.path.join(downloaded_images_path, img_file_name)
    img_to_check = cv2.imread(img_path)
    
    if img_to_check is not None:
        detected_faces = parfois_eye_detector_2.detect_faces(img_to_check)
        
        for face in detected_faces:
            if "keypoints" in face and "left_eye" in face["keypoints"] and "right_eye" in face["keypoints"] and "mouth_left" in face["keypoints"] and "mouth_right" in face["keypoints"]:
                return 1 
            if "keypoints" in face and "left_eye" in face["keypoints"] and "mouth_left" in face["keypoints"]:
                return 1  
            if "keypoints" in face and "right_eye" in face["keypoints"] and "mouth_right" in face["keypoints"]:
                return 1  
        return 0
    else:
        print(f"{img_file_name}: Unable to read the image")
        return None

images_to_check["model_images_predictions"] = images_to_check.apply(model_predictions_2, axis=1)

1/1 [==============================] - 0s 20ms/step


In [16]:
studio_folder_true = pd.read_excel(target_folder3)
results_check = (images_to_check["model_images_predictions"] != studio_folder_true["model_images_check"]).sum()
results_check

10

##### After the check i got again 10 wrong predictions. Its not performing differently but i believe we should keep the keypoin analysis. 

In [23]:
images_to_check["model_images_predictions"] = np.nan

### 2.4 Model Run (With more Param. Tuning and Keypoint targeting)

#### Version three

##### I changed another set of parameters.

##### scale_factor determines the ratio between consecutive scales in the image pyramid. In simple terms, for a database of small images, this needs to be smaller. Also if we decrease this parameter, we can get better results, but far more needed computational power.

##### Also, i will target only the eyes, right or left.

In [28]:
parfois_eye_detector_3 = MTCNN(min_face_size=30, 
                               steps_threshold=[0.7, 0.8, 0.8],
                               scale_factor=0.1)

def model_predictions_2(row):
    img_file_name = row["image_file_name"]
    img_path = os.path.join(downloaded_images_path, img_file_name)
    img_to_check = cv2.imread(img_path)
    
    if img_to_check is not None:
        detected_faces = parfois_eye_detector_3.detect_faces(img_to_check)
        
        for face in detected_faces:
            if "keypoints" in face and "left_eye" in face["keypoints"] and "right_eye" in face["keypoints"] and "mouth_left" in face["keypoints"] and "mouth_right" in face["keypoints"]:
                return 1 
            if "keypoints" in face and "left_eye" in face["keypoints"] and "mouth_left" in face["keypoints"]:
                return 1  
            if "keypoints" in face and "right_eye" in face["keypoints"] and "mouth_right" in face["keypoints"]:
                return 1  
            if "keypoints" in face and "right_eye" in face["keypoints"]:
                return 1  
            if "keypoints" in face and "left_eye" in face["keypoints"]:
                return 1  
        return 0
    else:
        print(f"{img_file_name}: Unable to read the image")
        return None

images_to_check["model_images_predictions"] = images_to_check.apply(model_predictions_2, axis=1)

1/1 [==============================] - 0s 21ms/step


In [29]:
studio_folder_true = pd.read_excel(target_folder3)
results_check = (images_to_check["model_images_predictions"] != studio_folder_true["model_images_check"]).sum()
results_check

20

##### I loose correctness if we change the scale_factor, but keep the performance if we keep the left_eye right_eye only tracking. 


### 2.4 Model Run (Final Version)

In [30]:
parfois_eye_detector_4 = MTCNN(min_face_size=30, 
                               steps_threshold=[0.7, 0.8, 0.8])

def model_predictions_2(row):
    img_file_name = row["image_file_name"]
    img_path = os.path.join(downloaded_images_path, img_file_name)
    img_to_check = cv2.imread(img_path)
    
    if img_to_check is not None:
        detected_faces = parfois_eye_detector_4.detect_faces(img_to_check)
        
        for face in detected_faces:
            if "keypoints" in face and "left_eye" in face["keypoints"] and "right_eye" in face["keypoints"] and "mouth_left" in face["keypoints"] and "mouth_right" in face["keypoints"]:
                return 1 
            if "keypoints" in face and "left_eye" in face["keypoints"] and "mouth_left" in face["keypoints"]:
                return 1  
            if "keypoints" in face and "right_eye" in face["keypoints"] and "mouth_right" in face["keypoints"]:
                return 1  
            if "keypoints" in face and "right_eye" in face["keypoints"]:
                return 1  
            if "keypoints" in face and "left_eye" in face["keypoints"]:
                return 1  
        return 0
    else:
        print(f"{img_file_name}: Unable to read the image")
        return None

images_to_check["model_images_predictions"] = images_to_check.apply(model_predictions_2, axis=1)

1/1 [==============================] - 0s 19ms/step


In [31]:
studio_folder_true = pd.read_excel(target_folder3)
results_check = (images_to_check["model_images_predictions"] != studio_folder_true["model_images_check"]).sum()
results_check

10